Forked the script so i just add the lower graph
------

my analysis of the problem of mercedes is here...btw

https://www.kaggle.com/plarmuseau/variance-analysis-ii
Quoting myself...
> Some Aftertoughts... It seems to me that mercedes asked the wrong
> question. They asked to forecast the production. What they got is the
> prove you can't forecast that variability. They should have asked: how
> can we minimize that variability, and thats a whole other answer.
> Thats reshuffling the production in such a way that you get away with
> the bottlenecks and diversify away the production risks...

SVR with Categorical Kernel Matrix
----------------------------------

*Summary*: a model using sklearn.svm.SVR(kernel='precomputed')

Support Vector Regression seems appropriate for this competition, it is robust against outliers and works best on small training sets.

With just 4209 training rows it is feasible to compute a pairwise similarity metric between all rows, requiring only 4209*4209 = 17,715,681 entries. (Even using 8 byte floats, this is only 142Mb.)

This means we can precompute a kernel matrix over all the data. The SVR then learns not from a collection of feature vectors, but from a training set that is defined entirely by similarities to training rows. The model is a set of support vectors (training instances) and a weight for each.

Given that the training rows are entirely categorical, a categorical kernel is needed.

I've experimented with just the simplest categorical kernel - a count of matching categories in a (small) subset of columns, typically 3-8, always including X0 and X5.

This simple categorical kernel below is in fact the same as using a linear kernel on a one hot coded feature set. A linear SVR's dot product between one hot feature vectors would result in the same 'similarity score' as the calc_gram function computes (but unnormalised).

With more domain knowledge, interesting non-linearities could be added to the kernel function, for example a 'gating' mechanism, whereby if a particular original feature column does not match, it multiplies the similarity metric by a discount factor, shrinking the similarity, making a match in that column more important.

Hypothetically, the boolean X10+ attributes in the data could be representing some kind of branching tree structure of vehicle tests, so maybe tree kernels would be an even better way of representing the space of training examples.

There are some good further ideas in "An investigation into new kernels for categorical variables" here:
https://upcommons.upc.edu/handle/2099.1/17172?show=full

For example, using the probability distribution of a column to make matching rare categories more similar than matching common categories. Also, using the product of column similarities instead of the mean.

I find this interesting, because it is a totally different way of learning, there are possibilities for adding domain knowledge in the form of a custom kernel function that would be hard to encode in regular columns of features. The standard ML technique of summarizing lists or tree structured data down into single columns seems like it could easily lose an important property of the data.

On the other hand, it does simply transfer the work from feature engineering into 'kernel function engineering'.

For this competition, my CV scores from this model (with some added experimental non-linearity) were in the 0.575 - 0.577 R2 region, varying with feature sets, and a 'standard' public LB 0.556, private LB 0.550 range, and highly consistent. Like everyone else's results, not a very significant improvement on a mean-y given X0 lookup table model.

In [ ]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
    
def load_numeric_csv(name):
	df = pd.read_csv(name)
	for c in df.columns:
		if df[c].dtype=='object':
			df[c] = df[c].map(lambda x: int(x,36))
	return df

# return one column in a 2D array
def get_col(df, col):
	return np.reshape(df[col].values, (-1,1))

# Compute simple categorical similarity, returns M*N matrix
#  - if categories same, similarity = 1
#  - if categories different, similarity = 0
def binary_similarity(df, df2, col):
	return (pairwise_distances(get_col(df, col), get_col(df2, col), metric='l1') == 0).astype(float)

# Compute simple categorical similarity matrix over a set of columns.
# This is just a normalised sum of the single column similarities...
# i.e. a normalised count of category matches in cols.
# With more domain knowledge, nonlinearities can be added here...
def calc_gram(df, df2, cols):
	gram = binary_similarity(df, df2, cols[0])
	for i, c in enumerate(cols[1:]):
		d = binary_similarity(df, df2, c)
		gram += d
	return gram / len(cols)  # normalise to 0..1

def score_summary_verbose(y, p):
	return 'mean y %9.4f %4d  rmse %9.6f  r2 %.6f '%(np.mean(y), len(y), mean_squared_error(y, p)**.5, r2_score(y, p))

def score_summary(y, p):
	return '[rmse %9.6f r2 %.6f]'%(mean_squared_error(y, p)**.5, r2_score(y, p))

# run k-fold cross validation on the data, returning oof predictions
def run_kfold(df, y, cols, C, epsilon, folds=10):
    print('using', df.shape, 'outliers:', np.sum(y>200))
    print('C=%.1f; epsilon=%.2f; using %d input cols; %s'%(C, epsilon, len(cols), ':'.join(cols)))

    oof = np.zeros(df.shape[0])
    kf = KFold(n_splits=folds, shuffle=True, random_state=42).split(df)
    for i, (train_index, valid_index) in enumerate(kf):
        x_tr = df.loc[train_index]
        y_tr = y[train_index]
        x_va = df.loc[valid_index]
        y_va = y[valid_index]

        gram = calc_gram(x_tr, x_tr, cols)
        clf = svm.SVR(kernel='precomputed', C=C, epsilon=epsilon)
        clf.fit(gram, y_tr)

        # predict on training examples
        p = clf.predict(gram)
        s1 = score_summary(y_tr, p)

        # recompute kernel matrix on validation fold vs training fold
        gram = calc_gram(x_va, x_tr, cols)
        # predict on validation examples
        p = clf.predict(gram)
        s2 = score_summary(y_va, p)
        oof[valid_index] = p

        # Last value shows # of support vectors from training set being used
        print(i, 'Train:', s1, ' Valid:', s2, clf.support_.shape)

    return oof

Now load the training set, and try cross validation on a simple set of features.

In [ ]:
train = load_numeric_csv("../input/train.csv")

# similar to Raddar's features here:
# https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/discussion/36136
cols = ['X0', 'X5','X47']

# Adapted from a simple grid search over regularisation settings...
# You can change these to experiment
for C in [ 200 ]:
    for epsilon in [ 6 ]:
        oof = run_kfold(train, train.y, cols, C, epsilon)
        print('Result for C %.1f epsilon %.2f:  R2 %.6f'%(C, epsilon, r2_score(train.y, oof)))

Let's visualize the out of fold predictions...

In [ ]:
# adapted from https://www.kaggle.com/plarmuseau/linear-programming-solution
plt.figure(figsize=(8,8))
plt.title('True vs. Pred. train')
plt.plot([80,265], [80,265], color='g', alpha=0.3)
plt.scatter(x=train.y, y=oof, marker='.', alpha=0.5)
plt.scatter(x=[np.mean(train.y)], y=[np.mean(oof)], marker='o', color='red')
plt.xlabel('Real train');
plt.ylabel('Pred. train')


THIS GRAPH was something we tried to fit until it collided, this solution we considered as bad...
------
They look very familiar...

Try making a submission, training on the whole training set and predicting on the test set.

In [ ]:
def make_sub(train, y, test, cols, C, epsilon):
    print('C=%.1f; epsilon=%.2f; using %d input cols; %s'%(C, epsilon, len(cols), ':'.join(cols)))

    gram = calc_gram(train, train, cols)
    clf = svm.SVR(kernel='precomputed', C=C, epsilon=epsilon)
    clf.fit(gram, y)

    # predict on training examples
    p = clf.predict(gram)
    print('Submission training error:', score_summary(y,p), clf.support_.shape)

    # recompute kernel matrix on test vs train
    gram = calc_gram(test, train, cols)
    # predict on test examples
    p = clf.predict(gram)
    
    filename = 'SVR_%s_%.1f_%.2f.csv'%('.'.join(cols), C, epsilon)
    print('unique predictions:', len(np.unique(p)))
    print(pd.Series(p).describe())
    print('saving to', filename)

    sub = pd.DataFrame()
    sub['ID'] = test.ID
    sub['y'] = p
    sub.to_csv(filename, index=False)
    return sub['y']
    
test = load_numeric_csv("../input/test.csv")
y_pred=make_sub(train, train.y, test, cols, 200, 6)

In [ ]:
plt.figure(figsize=(16,4))

plt.subplot(1,4,1)
#train_scores = cv_result['train-rmse']
#train_stds = cv_result['train-r2-std']
#plt.plot(train_scores, color='green')
#plt.fill_between(range(len(cv_result)), train_scores - train_stds, train_scores + train_stds, alpha=0.1, color='green')
#test_scores = cv_result['train-rmse']
#test_stds = cv_result['test-r2-std']
#plt.plot(test_scores, color='red')
#plt.fill_between(range(len(cv_result)), test_scores - test_stds, test_scores + test_stds, alpha=0.1, color='red')
plt.title('Train and test cv scores (R2)')

plt.subplot(1,4,2)
plt.title('True vs. Pred. train')
plt.plot([80,265], [80,265], color='g', alpha=0.3)
plt.scatter(x=train.y, y=oof, marker='.', alpha=0.5)
plt.scatter(x=[np.mean(train.y)], y=[np.mean(oof)], marker='o', color='red')
plt.xlabel('Real train'); plt.ylabel('Pred. train')

plt.subplot(1,4,3)
sns.distplot(train.y, kde=False, color='g')
sns.distplot(oof, kde=False, color='r')
plt.title('Distr. of train and pred. train')

plt.subplot(1,4,4)
sns.distplot(train.y, kde=False, color='g')
sns.distplot(y_pred, kde=False, color='b')
plt.title('Distr. of train and pred. test')



plt.figure(figsize=(18,1))
plt.plot( train.y[:200], color='r', linewidth=0.7)
plt.plot(y_pred[:200], color='g', linewidth=0.7)
plt.title('First 200 true and pred. trains')

print('Mean error =', np.mean(train.y - oof))
print('Train r2 =', r2_score(train.y, oof))

Here you see how random the prediction is;.. prediction versus trained...
-----------------

That is it! A few seconds to compute the kernel matrix, a few seconds to train, and a few seconds to predict :)